# Django ORM - Praca z danymi (komendy DML i DQL)

**Definicje modelu**

<code>class Task(models.Model):
    name = models.CharField(max_length=64)
</code>

In [1]:
from flask import Flask
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy.orm import DeclarativeBase

class Base(DeclarativeBase):
  pass

db = SQLAlchemy(model_class=Base)

# create the app
app = Flask(__name__)

# configure the SQLite database, relative to the app instance folder
app.config["SQLALCHEMY_DATABASE_URI"] = "sqlite:///db.sqlite3"

# initialize the app with the extension
db.init_app(app)

from sqlalchemy import Integer, String
from sqlalchemy.orm import Mapped, mapped_column

class Task(db.Model):
    id: Mapped[int] = mapped_column(Integer, primary_key=True)
    name: Mapped[str] = mapped_column(String)

app.app_context().push()

## R - Read (CRUD) - DQL (Data Query Language) cd

### Funkcja agregujące

Funkcje agregujące w sql to operatory, które wykonują na wskazanych danych proste statystyki takie jak średnia, wartość maksymalna, suma czy liczba wpisów (SUM, AVG, MIN, MAX, COUNT). W Django funkcje agregujące realizowane są przez metodę aggregate. Metoda aggregate jako parametr przyjmuje wywołanie odpowiedniej klasy z modułu django.db.models (Sum, Avg, Min, Max, Count, ...).

Znajdź sumę indeksów wszystkich wpisów tabeli Task. 

In [10]:
from django.db.models import Sum

a_sum = Task.objects.aggregate(Sum('id'))
print(a_sum)

{'id__sum': 55}


Funkcja agregująca zwraca słownik z kluczem 
< nazwa_kolumny_wzdluz_ktorej_zachodzi_agregacja >_ _<nazwa_funkcji_agregującej> oraz wartością będąca wyliczoną statystyką.

Znajdź średnią wartość indeksu wpisów tabeli Task o wartości w kolumnie name "Szukanie"

In [11]:
from django.db.models import Avg

avg = Task.objects.filter(name="Szukanie").aggregate(Avg('id'))
print(avg)

{'id__avg': 5.5}


## U - UPDATE (CRUD) - DML (Data Manipulation Language)

### Klauzula UPDATE

In [4]:
# Metoda I - metoda update klasy Query
# UWAGA! Dane należy najperw przefiltrować, żeby jednym zapytanie NIE ZMIENIĆ WSZYSTKICH wpisów 
# w tabeli.

tasks = Task.query.filter_by(name="Szukanie").update({'name': 'Znajdywanie'})
print(tasks)
db.session.commit()

0


Widzimy, że metoda update nie zwraca nam obiektu klasy QuerySet tylko liczbę zmodyfikowanych wpisów.

In [6]:
# Metoda II - bezpośrednia modyfikacja wartości atrybutu instancji modelu

task = Task.query.filter_by(name="Znajdywanie").first()
task.name = "Szukanie"
db.session.commit()


## D - DELETE (CRUD) - DML (DATA Manipulation Language)

### Klauzula DELETE

In [8]:
# Metoda I - metoda delete klasy Query
# UWAGA! Dane należy najperw przefiltrować, żeby jednym zapytanie NIE USUNĄĆ WSZYSTKICH wpisów 
# w tabeli.

task = Task.query.filter_by(name="Pływanie").delete()
print(task)
db.session.commit()

0


Podobnie jak metoda update, metoda delete nie zwraca nam obiektu klasy Query tylko informacje o liczbie usuniętych wpisów

In [46]:
# Można też w inny sposób

task = Task.query.filter_by(name="Szukanie").first()
db.session.delete(task)
db.session.commit()

<Task 6>


W ten sposób wywołujemy polecenia DML i DDL za pomocą SQLAlchemy ORM. 

To samo możemy zrobić za pomocą SQLAlchemy Core.

## Core

Kilka przykładów

In [47]:
# C z CRUD
from sqlalchemy import insert

stmt = (
    insert(Task).
    values(name='Suszenie')
)
db.session.execute(stmt)
db.session.commit()

In [48]:
# R Z CRUD
from sqlalchemy import select

stmt = select(Task)
print(stmt)

SELECT task.id, task.name 
FROM task


In [49]:
result = db.session.execute(stmt)

for item in result:
    print(item)

(<Task 1>,)
(<Task 2>,)
(<Task 3>,)
(<Task 4>,)
(<Task 7>,)
(<Task 8>,)
(<Task 9>,)
(<Task 10>,)
(<Task 11>,)


In [50]:
# fetchall

result = db.session.execute(stmt)
print(result.fetchall())

[(<Task 1>,), (<Task 2>,), (<Task 3>,), (<Task 4>,), (<Task 7>,), (<Task 8>,), (<Task 9>,), (<Task 10>,), (<Task 11>,)]


In [51]:
# fetchmany

result = db.session.execute(stmt)
print(result.fetchmany(size=5))

[(<Task 1>,), (<Task 2>,), (<Task 3>,), (<Task 4>,), (<Task 7>,)]


In [52]:
# fetchaone

result = db.session.execute(stmt)
print(result.fetchone())

(<Task 1>,)


### Filtry

In [54]:
stmt = (
    select(Task.name).
    where(Task.id==4)
)
task = db.session.execute(stmt)
print(task.fetchall())

[('Malowanie',)]


In [56]:
stmt = (
    select(Task).
    filter_by(name="Szukanie")
)
task = db.session.execute(stmt).fetchall()
print(task)

[]


In [58]:
# U z CRUD
from sqlalchemy import update
stmt = (
    update(Task).
    where(Task.id == 2).
    values(name='Odśnieżanie')
)
db.session.execute(stmt)
db.session.commit()

In [59]:
# D Z CRUD
from sqlalchemy import delete

stmt = (
    delete(Task).
    where(Task.id == 2)
)
db.session.execute(stmt)
db.session.commit()